https://www.youtube.com/watch?v=KYFfMymTspE

In [7]:
from keras.applications.nasnet import NASNetMobile
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator 
from keras import backend as K

In [8]:
img_rows,img_cols = 224, 224

nasnet = NASNetMobile(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

for layer in nasnet.layers:
    layer.trainable=False

def addTopModelnasnet(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 7

FC_Head = addTopModelnasnet(nasnet,num_classes)

model = Model(inputs = nasnet.input, outputs = FC_Head)

print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 111, 111, 32) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

adjust_conv_projection_12 (Conv (None, 7, 7, 176)    185856      activation_364[0][0]             
__________________________________________________________________________________________________
normal_conv_1_12 (Conv2D)       (None, 7, 7, 176)    185856      activation_365[0][0]             
__________________________________________________________________________________________________
adjust_bn_12 (BatchNormalizatio (None, 7, 7, 176)    704         adjust_conv_projection_12[0][0]  
__________________________________________________________________________________________________
normal_bn_1_12 (BatchNormalizat (None, 7, 7, 176)    704         normal_conv_1_12[0][0]           
__________________________________________________________________________________________________
activation_366 (Activation)     (None, 7, 7, 176)    0           normal_bn_1_12[0][0]             
__________________________________________________________________________________________________
activation

In [9]:
train_data_dir = 'C:\\Users\\user\\AI_vision\\project_carrot\\CNN\\data\\train_skeleton'
validation_data_dir = 'C:\\Users\\user\\AI_vision\\project_carrot\\CNN\\data\\test_skeleton'
# 6:2:2로 나눠야 하지 않아?

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'finger_sign_NASNetMobile.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=10,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 1200
nb_validation_samples = 300  

epochs = 20
batch_size = 32

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)


Found 4165 images belonging to 7 classes.
Found 1785 images belonging to 7 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
37/37 [==============================] - 156s 4s/step - loss: 1.5303 - acc: 0.3936 - val_loss: 1.2782 - val_acc: 0.5590

Epoch 00001: val_loss improved from inf to 1.27819, saving model to finger_sign_NASNetMobile.h5
Epoch 2/20
37/37 [==============================] - 140s 4s/step - loss: 1.0410 - acc: 0.5270 - val_loss: 1.2543 - val_acc: 0.4340

Epoch 00002: val_loss improved from 1.27819 to 1.25431, saving model to finger_sign_NASNetMobile.h5
Epoch 3/20
37/37 [==============================] - 142s 4s/step - loss: 0.9004 - acc: 0.5684 - val_loss: 2.2701 - val_acc: 0.2535

Epoch 00003: val_loss did not improve from 1.25431
Epoch 4/20
37/37 [==============================] - 134s 4s/step - loss: 0.8155 - acc: 0.6319 - val_loss: 1.6932 - val_acc: 0.2465

Epoch 00004: val_loss did not improve from 1.25431
Epoch 5/20
37/37 [========================